In [152]:
%%writefile constants.cuh
#ifndef CONSTANTS_H
#define CONSTANTS_H

#define WINDOW_WIDTH 800
#define WINDOW_HEIGHT 600
#define PARTICLE_RADIUS 5.0f

#define MAX_LEFT PARTICLE_RADIUS
#define MAX_RIGHT (WINDOW_WIDTH - PARTICLE_RADIUS)
#define MAX_TOP PARTICLE_RADIUS
#define MAX_BOTTOM (WINDOW_HEIGHT - PARTICLE_RADIUS)

#define GRAVITY 250.0f
#define DAMPING_FACTOR 0.9f

#define GRID_CELL_SIZE (PARTICLE_RADIUS * 2.0f)
#define GRID_WIDTH (WINDOW_WIDTH / GRID_CELL_SIZE)
#define GRID_HEIGHT (WINDOW_HEIGHT / GRID_CELL_SIZE)
#define MAX_PARTICLES_PER_CELL 16

#endif // CONSTANTS_H


Overwriting constants.cuh


In [153]:
%%writefile collisions.cuh
#ifndef COLLISIONS_H
#define COLLISIONS_H

#include "constants.cuh"
#include <cuda_runtime.h>


#ifdef __cplusplus
extern "C" {
#endif

typedef struct {
    float2 position;
    float2 velocity;
    float  radius;
} Particle;


typedef struct {
    float2 position;
    float radius;
} Obstacle;


void runCollisionStep(Particle *d_particles, int *d_gridCounters, int *d_gridIndices, int numParticles);

#ifdef __cplusplus
}
#endif

#endif // COLLISIONS_H


Overwriting collisions.cuh


In [154]:
%%writefile collisions.cu
#include "collisions.cuh"
#include "constants.cuh"
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <math.h>

// Collision resolution kernel: For each particle, check its neighborhood in the grid
// and resolve any overlaps with other particles.
__global__ void resolveCollisionsKernel(Particle *particles, int *gridCounters, int *gridIndices, int numParticles) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= numParticles) return;

    Particle p = particles[i];

    // Determine particle's cell
    int cellX = (int)(p.position.x / GRID_CELL_SIZE);
    int cellY = (int)(p.position.y / GRID_CELL_SIZE);

    // Loop over 3x3 neighborhood
    for (int offsetY = -1; offsetY <= 1; offsetY++) {
        for (int offsetX = -1; offsetX <= 1; offsetX++) {
            int neighborX = cellX + offsetX;
            int neighborY = cellY + offsetY;

            // Skip if outside grid
            if (neighborX < 0 || neighborX >= GRID_WIDTH ||
                neighborY < 0 || neighborY >= GRID_HEIGHT)
                continue;

            int neighborIndex = neighborY * GRID_WIDTH + neighborX;
            int count = gridCounters[neighborIndex];

            for (int j = 0; j < count; j++) {
                int otherIndex = gridIndices[neighborIndex * MAX_PARTICLES_PER_CELL + j];
                if (otherIndex == i) continue;

                Particle other = particles[otherIndex];

                float dx = other.position.x - p.position.x;
                float dy = other.position.y - p.position.y;
                float distance = sqrtf(dx * dx + dy * dy);
                float minDist = p.radius + other.radius;

                if (distance < minDist && distance > 0.0f) {
                    // Normalize collision normal
                    float normX = dx / distance;
                    float normY = dy / distance;

                    // Push particles apart
                    float overlap = (minDist - distance) * 0.5f;
                    p.position.x -= overlap * normX;
                    p.position.y -= overlap * normY;

                    // Reflect velocity
                    float vDotN = p.velocity.x * normX + p.velocity.y * normY;
                    if (vDotN > 0.0f) vDotN = 0.0f;

                    p.velocity.x -= 2.0f * vDotN * normX;
                    p.velocity.y -= 2.0f * vDotN * normY;

                    // Dampen
                    p.velocity.x *= DAMPING_FACTOR;
                    p.velocity.y *= DAMPING_FACTOR;

                    // Wake-up nudge if velocity is too low
                    float speed2 = p.velocity.x * p.velocity.x + p.velocity.y * p.velocity.y;
                    if (speed2 < 1e-4f) {

                        p.velocity.x += 0.1f * normX;
                        p.velocity.y += 0.1f * normY;
                    }
                }
            }
        }
    }

    float jitter = 0.05f;
    p.velocity.x += jitter * ((i % 2 == 0) ? 1.0f : -1.0f); // pseudo-random X
    p.velocity.y += jitter * ((i % 3 == 0) ? 1.0f : -1.0f); // pseudo-random Y

    particles[i] = p;
}

__global__ void resolveCollisionsKernelSymmetric(Particle *particles,
                                                   int *gridCounters,
                                                   int *gridIndices,
                                                   int numParticles) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= numParticles) return;

    // Load particle i (local copy is ok for reading cell info)
    Particle p = particles[i];

    // Determine particle's cell
    int cellX = (int)(p.position.x / GRID_CELL_SIZE);
    int cellY = (int)(p.position.y / GRID_CELL_SIZE);

    // Loop over the 3x3 neighborhood of cells.
    for (int offsetY = -1; offsetY <= 1; offsetY++) {
        for (int offsetX = -1; offsetX <= 1; offsetX++) {
            int neighborX = cellX + offsetX;
            int neighborY = cellY + offsetY;

            // Skip if neighbor cell is outside the grid.
            if (neighborX < 0 || neighborX >= GRID_WIDTH ||
                neighborY < 0 || neighborY >= GRID_HEIGHT)
                continue;

            int neighborCell = neighborY * GRID_WIDTH + neighborX;
            int count = gridCounters[neighborCell];

            // Check each particle in the neighbor cell.
            for (int j = 0; j < count; j++) {
                int otherIndex = gridIndices[neighborCell * MAX_PARTICLES_PER_CELL + j];

                // Ensure each pair is processed only once.
                if (otherIndex <= i)
                    continue;

                // Read the neighbor particle.
                Particle other = particles[otherIndex];

                float dx = other.position.x - p.position.x;
                float dy = other.position.y - p.position.y;
                float distance = sqrtf(dx * dx + dy * dy);
                float minDist = p.radius + other.radius;

                if (distance < minDist && distance > 0.0f) {
                    // Calculate normalized collision normal.
                    float normX = dx / distance;
                    float normY = dy / distance;

                    // Compute symmetric overlap correction.
                    float overlap = (minDist - distance) * 0.5f;
                    atomicAdd(&particles[i].position.x, -overlap * normX);
                    atomicAdd(&particles[i].position.y, -overlap * normY);
                    atomicAdd(&particles[otherIndex].position.x, overlap * normX);
                    atomicAdd(&particles[otherIndex].position.y, overlap * normY);

                    // Compute relative velocity along the collision normal.
                    float relVel = (other.velocity.x - p.velocity.x) * normX +
                                   (other.velocity.y - p.velocity.y) * normY;

                    // Only process if particles are moving toward each other.
                    if (relVel < 0.0f) {
                        // Use DAMPING_FACTOR as the restitution coefficient.
                        float restitution = DAMPING_FACTOR;
                        float impulse = -(1.0f + restitution) * relVel / 2.0f; // Equal mass assumed

                        // Atomically update velocities for both particles.
                        atomicAdd(&particles[i].velocity.x, -impulse * normX);
                        atomicAdd(&particles[i].velocity.y, -impulse * normY);
                        atomicAdd(&particles[otherIndex].velocity.x, impulse * normX);
                        atomicAdd(&particles[otherIndex].velocity.y, impulse * normY);
                    }
                }
            }
        }
    }
}



// Host function to launch the collision resolution kernel.
void runCollisionStep(Particle *d_particles, int *d_gridCounters, int *d_gridIndices, int numParticles) {
    int threadsPerBlock = 256;
    int numBlocks = (numParticles + threadsPerBlock - 1) / threadsPerBlock;
    resolveCollisionsKernelSymmetric<<<numBlocks, threadsPerBlock>>>(d_particles, d_gridCounters, d_gridIndices, numParticles);
}


Overwriting collisions.cu


In [155]:
%%writefile simulation.cuh
#ifndef SIMULATION_H
#define SIMULATION_H

#include "collisions.cuh"  // This header now defines Particle.
#include "constants.cuh"
#include <cuda_runtime.h>

#ifdef __cplusplus
extern "C" {
#endif

// Run a full simulation step (integration, grid build, collision resolution).
void runSimulationStep(Particle *d_particles, int numParticles, float deltaTime, float gravity,
                       int *d_gridCounters, int *d_gridIndices, Obstacle* obstacles,
    int numObstacles);
#ifdef __cplusplus
}
#endif

#endif // SIMULATION_H

Overwriting simulation.cuh


In [156]:
%%writefile simulation.cu
#include "simulation.cuh"
#include "collisions.cuh"  // Include the collision module.
#include "constants.cuh"
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <math.h>

// Integration kernel: Updates particle velocity (with gravity) and position.
__global__ void integrateParticlesKernel(Particle *particles, int numParticles, float deltaTime, float gravity,
                                         Obstacle* obstacles, int numObstacles) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= numParticles) return;

    Particle p = particles[i];

    // Gravity
    p.velocity.y += gravity * deltaTime;

    // Integrate motion
    p.position.x += p.velocity.x * deltaTime;
    p.position.y += p.velocity.y * deltaTime;

    // Obstacle collisions
    for (int o = 0; o < numObstacles; o++) {
        Obstacle obs = obstacles[o];
        float dx = p.position.x - obs.position.x;
        float dy = p.position.y - obs.position.y;
        float dist = sqrtf(dx * dx + dy * dy);
        float minDist = p.radius + obs.radius;

        if (dist < minDist && dist > 0.0f) {
            float normX = dx / dist;
            float normY = dy / dist;
            float overlap = minDist - dist;

            p.position.x += normX * overlap;
            p.position.y += normY * overlap;

            float vDotN = p.velocity.x * normX + p.velocity.y * normY;
            if (vDotN < 0.0f) {
                p.velocity.x -= 2.0f * vDotN * normX;
                p.velocity.y -= 2.0f * vDotN * normY;
            }

            p.velocity.x *= DAMPING_FACTOR;
            p.velocity.y *= DAMPING_FACTOR;
        }
    }

    // Boundary collision
    if (p.position.x < MAX_LEFT) {
        p.position.x = MAX_LEFT;
        p.velocity.x = -p.velocity.x * DAMPING_FACTOR;
    }
    if (p.position.x > MAX_RIGHT) {
        p.position.x = MAX_RIGHT;
        p.velocity.x = -p.velocity.x * DAMPING_FACTOR;
    }
    if (p.position.y < MAX_TOP) {
        p.position.y = MAX_TOP;
        p.velocity.y = -p.velocity.y * DAMPING_FACTOR;
    }
    if (p.position.y > MAX_BOTTOM) {
        p.position.y = MAX_BOTTOM;
        p.velocity.y = -p.velocity.y * DAMPING_FACTOR;
    }

    particles[i] = p;
}

// Grid building kernel: Bins particles into a uniform grid using atomic operations.
__global__ void buildGridKernel(Particle *particles, int numParticles,
                                int *gridCounters, int *gridIndices) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= numParticles) return;

    Particle p = particles[i];
    int cellX = (int)(p.position.x / GRID_CELL_SIZE);
    int cellY = (int)(p.position.y / GRID_CELL_SIZE);

    cellX = min(max(cellX, 0), static_cast<int>(GRID_WIDTH) - 1);
    cellY = min(max(cellY, 0), static_cast<int>(GRID_HEIGHT) - 1);


    int cellIndex = cellY * GRID_WIDTH + cellX;

    int offset = atomicAdd(&gridCounters[cellIndex], 1);
    if (offset < MAX_PARTICLES_PER_CELL) {
        gridIndices[cellIndex * MAX_PARTICLES_PER_CELL + offset] = i;
    }
}

// Host function that performs one simulation step.
void runSimulationStep(Particle *d_particles, int numParticles, float deltaTime, float gravity,
                       int *d_gridCounters, int *d_gridIndices, Obstacle* obstacles,
    int numObstacles) {
    int threadsPerBlock = 256;
    int numBlocks = (numParticles + threadsPerBlock - 1) / threadsPerBlock;

    // 1. Integration: update particle positions and velocities.
    integrateParticlesKernel<<<numBlocks, threadsPerBlock>>>(
        d_particles, numParticles, deltaTime, gravity, obstacles, numObstacles);

    // 2. Reset the grid counters.
    int numCells = GRID_WIDTH * GRID_HEIGHT;
    cudaMemset(d_gridCounters, 0, numCells * sizeof(int));

    // 3. Build the uniform grid.
    buildGridKernel<<<numBlocks, threadsPerBlock>>>(d_particles, numParticles, d_gridCounters, d_gridIndices);

    // 4. Resolve collisions using the dedicated collision module.
    runCollisionStep(d_particles, d_gridCounters, d_gridIndices, numParticles);
}



Overwriting simulation.cu


In [157]:
%%writefile render.cuh
#ifndef RENDER_H
#define RENDER_H

#include "collisions.cuh"  // Uses Particle defined in collisions.h.
#include "constants.cuh"
#include <cuda_runtime.h>

#ifdef __cplusplus
extern "C" {
#endif

// Render function: writes particle colors to a framebuffer.
void renderParticles(const Particle* d_particles, int numParticles,
                     const Obstacle* d_obstacles, int numObstacles,
                     unsigned char* d_framebuffer, int width, int height, int frameNumber);

#ifdef __cplusplus
}
#endif

#endif // RENDER_H

Overwriting render.cuh


In [158]:
%%writefile render.cu
#include "render.cuh"
#include "constants.cuh"
#include <cuda_runtime.h>
#include <device_launch_parameters.h>


__global__ void renderParticlesKernel(const Particle* particles, int numParticles,
                                      const Obstacle* obstacles, int numObstacles,
                                      unsigned char* framebuffer, int width, int height,
                                      int frameNumber)
 {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;

    int pixelIdx = (y * width + x) * 4;

    // Clear pixel to black.
    framebuffer[pixelIdx + 0] = 0;
    framebuffer[pixelIdx + 1] = 0;
    framebuffer[pixelIdx + 2] = 0;
    framebuffer[pixelIdx + 3] = 255;

    for (int o = 0; o < numObstacles; o++) {
      Obstacle obs = obstacles[o];
      float dx = x - obs.position.x;
      float dy = y - obs.position.y;
      if (dx * dx + dy * dy <= obs.radius * obs.radius) {
          framebuffer[pixelIdx + 0] = 100; // dark gray
          framebuffer[pixelIdx + 1] = 100;
          framebuffer[pixelIdx + 2] = 100;
          framebuffer[pixelIdx + 3] = 255;
          return; // Don't draw particles here
      }
    }

    // Check each particle (naively) to see if it covers this pixel.
    for (int i = 0; i < numParticles; i++) {
        Particle p = particles[i];
        float dx = x - p.position.x;
        float dy = y - p.position.y;
        if ((dx * dx + dy * dy) <= (p.radius * p.radius)) {
            if (frameNumber >= 500) {
                // RED after frame 200
                framebuffer[pixelIdx + 0] = 255;
                framebuffer[pixelIdx + 1] = 0;
                framebuffer[pixelIdx + 2] = 0;
            } else {
                // BLUE before frame 200
                framebuffer[pixelIdx + 0] = 0;
                framebuffer[pixelIdx + 1] = 0;
                framebuffer[pixelIdx + 2] = 255;
            }
            break;
        }
    }
}

void renderParticles(const Particle* d_particles, int numParticles,
                     const Obstacle* d_obstacles, int numObstacles,
                     unsigned char* d_framebuffer, int width, int height, int frameNumber) {
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((width + 15) / 16, (height + 15) / 16);
    renderParticlesKernel<<<numBlocks, threadsPerBlock>>>(
        d_particles, numParticles,
        d_obstacles, numObstacles,
        d_framebuffer, width, height, frameNumber);
}


Overwriting render.cu


In [169]:
%%writefile main.cu
#include "simulation.cuh"
#include "render.cuh"
#include "constants.cuh"
#include "collisions.cuh"
#include <cuda_runtime.h>
#include <iostream>
#include <fstream>
#include <cstdlib>


void saveFramePPM(const char* filename, unsigned char* framebuffer, int width, int height) {
    std::ofstream ofs(filename, std::ios::binary);
    if (!ofs) {
        std::cerr << "Error: Could not open file " << filename << " for writing." << std::endl;
        return;
    }
    // Write the PPM header: P6, width, height, and max color value.
    ofs << "P6\n" << width << " " << height << "\n255\n";
    // Write pixel data: for each pixel, write only the first 3 bytes (RGB)
    for (int i = 0; i < width * height; i++) {
        ofs.write(reinterpret_cast<char*>(&framebuffer[i * 4]), 3);
    }
    ofs.close();
}

int main() {
    // Set the number of particles.
    const int numParticles = 128;
    size_t particlesSize = numParticles * sizeof(Particle);

    // Allocate and initialize host particle data.
    Particle *h_particles = new Particle[numParticles];
    for (int i = 0; i < numParticles; i++) {
        h_particles[i].position = make_float2(rand() % WINDOW_WIDTH, rand() % WINDOW_HEIGHT);
        h_particles[i].velocity = make_float2(((rand() % 200) / 100.0f) - 1.0f,
                                              ((rand() % 200) / 100.0f) - 1.0f);
        h_particles[i].radius = PARTICLE_RADIUS;
    }

    Obstacle h_obstacles[] = {
    { make_float2(400.0f, 300.0f), 40.0f },
    { make_float2(200.0f, 500.0f), 25.0f }
    };
    int numObstacles = sizeof(h_obstacles) / sizeof(Obstacle);

    Obstacle* d_obstacles;
    cudaMalloc(&d_obstacles, sizeof(h_obstacles));
    cudaMemcpy(d_obstacles, h_obstacles, sizeof(h_obstacles), cudaMemcpyHostToDevice);

    // Allocate device memory for particles.
    Particle *d_particles;
    cudaMalloc(&d_particles, particlesSize);
    cudaMemcpy(d_particles, h_particles, particlesSize, cudaMemcpyHostToDevice);

    // Allocate device memory for grid arrays.
    int numCells = GRID_WIDTH * GRID_HEIGHT;
    int *d_gridCounters, *d_gridIndices;
    cudaMalloc(&d_gridCounters, numCells * sizeof(int));
    cudaMalloc(&d_gridIndices, numCells * MAX_PARTICLES_PER_CELL * sizeof(int));

    // Allocate device framebuffer (RGBA) for rendering.
    int width = WINDOW_WIDTH, height = WINDOW_HEIGHT;
    size_t framebufferSize = width * height * 4 * sizeof(unsigned char);
    unsigned char *d_framebuffer;
    cudaMalloc(&d_framebuffer, framebufferSize);

    // Allocate host memory for the framebuffer.
    unsigned char *h_framebuffer = new unsigned char[framebufferSize];

    // Simulation loop: Render and save a sequence of frames.
    const int numFrames = 1000;

    for (int frame = 0; frame < numFrames; frame++) {
        float deltaTime = 0.016f; // ~60 fps

        // Run one simulation step: integration, grid building, and collision resolution.
        runSimulationStep(d_particles, numParticles, deltaTime, GRAVITY, d_gridCounters, d_gridIndices, d_obstacles, numObstacles);

        // Render particles into the device framebuffer.
        renderParticles(d_particles, numParticles,
                d_obstacles, numObstacles,
                d_framebuffer, width, height, frame);

        // Copy the framebuffer from device to host.
        cudaMemcpy(h_framebuffer, d_framebuffer, framebufferSize, cudaMemcpyDeviceToHost);

        // Save the current frame as a PPM image.
        char filename[64];
        sprintf(filename, "zframe_%03d.ppm", frame);
        saveFramePPM(filename, h_framebuffer, width, height);

        std::cout << "Saved " << filename << std::endl;
    }

    // Clean up CUDA memory.
    cudaFree(d_particles);
    cudaFree(d_gridCounters);
    cudaFree(d_gridIndices);
    cudaFree(d_framebuffer);

    // Clean up host memory.
    delete[] h_particles;
    delete[] h_framebuffer;

    return 0;
}


Overwriting main.cu


In [170]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 main.cu simulation.cu collisions.cu render.cu -o particleSim


In [171]:
!./particleSim

Saved zframe_000.ppm
Saved zframe_001.ppm
Saved zframe_002.ppm
Saved zframe_003.ppm
Saved zframe_004.ppm
Saved zframe_005.ppm
Saved zframe_006.ppm
Saved zframe_007.ppm
Saved zframe_008.ppm
Saved zframe_009.ppm
Saved zframe_010.ppm
Saved zframe_011.ppm
Saved zframe_012.ppm
Saved zframe_013.ppm
Saved zframe_014.ppm
Saved zframe_015.ppm
Saved zframe_016.ppm
Saved zframe_017.ppm
Saved zframe_018.ppm
Saved zframe_019.ppm
Saved zframe_020.ppm
Saved zframe_021.ppm
Saved zframe_022.ppm
Saved zframe_023.ppm
Saved zframe_024.ppm
Saved zframe_025.ppm
Saved zframe_026.ppm
Saved zframe_027.ppm
Saved zframe_028.ppm
Saved zframe_029.ppm
Saved zframe_030.ppm
Saved zframe_031.ppm
Saved zframe_032.ppm
Saved zframe_033.ppm
Saved zframe_034.ppm
Saved zframe_035.ppm
Saved zframe_036.ppm
Saved zframe_037.ppm
Saved zframe_038.ppm
Saved zframe_039.ppm
Saved zframe_040.ppm
Saved zframe_041.ppm
Saved zframe_042.ppm
Saved zframe_043.ppm
Saved zframe_044.ppm
Saved zframe_045.ppm
Saved zframe_046.ppm
Saved zframe_

In [172]:
import re, os,sys
import matplotlib.pyplot as plt
from IPython.display import Image, display,clear_output
import time
from PIL import Image as PILImage # import the PIL library

files = os.listdir("/content")
file_names = []
for file_ in files:
    if file_.endswith("ppm"):
      # sys.stdout.write(file_)
      # sys.stdout.flush()
      file_names.append(file_)


In [173]:
file_names.sort()

In [164]:
!pip install imageio

In [174]:
import imageio

frames = []
for file_ in file_names:
  if os.path.exists(file_):
      frames.append(imageio.imread(file_))

# Save as an animated GIF
output_gif = "particle_simulation.gif"
imageio.mimsave(output_gif, frames, duration=0.05)  # 50 ms per frame

print(f"Saved GIF: {output_gif}")

<ipython-input-174-9bb0e8018f34>:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(file_))


Saved GIF: particle_simulation.gif


In [175]:
import numpy as np
import imageio

gif = imageio.get_reader("particle_simulation.gif", memtest=False)
differences = []

# Iterate through frames using a for loop and gif.iter_data()
for i, frame in enumerate(gif.iter_data()):
    if i > 0:  # Skip the first frame for comparison
        frame1 = np.array(prev_frame)
        frame2 = np.array(frame)
        diff = np.sum(frame1 != frame2)
        differences.append(diff)
    prev_frame = frame  # Store current frame for next comparison


# Report
diff_count = len(differences)
for i, diff in enumerate(differences):
    print(f"GIF frame {i:03d} → {i+1:03d}: {diff} differing pixels")

if all(d == 0 for d in differences):
    print("\n⚠️ All GIF frames are identical.")
else:
    print("\n✅ The GIF shows motion between frames.")
    print(f"GIF has {diff_count} frames.")


GIF frame 000 → 001: 265 differing pixels
GIF frame 001 → 002: 99 differing pixels
GIF frame 002 → 003: 556 differing pixels
GIF frame 003 → 004: 669 differing pixels
GIF frame 004 → 005: 1310 differing pixels
GIF frame 005 → 006: 611 differing pixels
GIF frame 006 → 007: 1888 differing pixels
GIF frame 007 → 008: 933 differing pixels
GIF frame 008 → 009: 1873 differing pixels
GIF frame 009 → 010: 2043 differing pixels
GIF frame 010 → 011: 1625 differing pixels
GIF frame 011 → 012: 1906 differing pixels
GIF frame 012 → 013: 2230 differing pixels
GIF frame 013 → 014: 2421 differing pixels
GIF frame 014 → 015: 2527 differing pixels
GIF frame 015 → 016: 2710 differing pixels
GIF frame 016 → 017: 2933 differing pixels
GIF frame 017 → 018: 3071 differing pixels
GIF frame 018 → 019: 3113 differing pixels
GIF frame 019 → 020: 3177 differing pixels
GIF frame 020 → 021: 3537 differing pixels
GIF frame 021 → 022: 3487 differing pixels
GIF frame 022 → 023: 3772 differing pixels
GIF frame 023 → 02

In [176]:
for file_ in file_names:
  if os.path.exists(file_):
      os.remove(file_)

files = os.listdir("/content")
file_names = []
for file_ in files:
    if file_.endswith("png"):
      os.remove(file_)

In [177]:
from google.colab import files
files.download('/content/particle_simulation.gif')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>